In [1]:
import IsingLib
import numpy as np
np.set_printoptions(threshold=1000)
np.set_printoptions(linewidth=200)

# Random Graph

num = 15
value = 31
mat = np.random.randint(-value, value, (num, num))
print(mat)

[[-10 -11  10 -16  -8  -7  -5   3  13 -24   4  -5  -5 -24 -12]
 [ 26  -6  20  -5   2  14 -28  -4   9   8  25 -24   9  18  -4]
 [ 14   8 -27  -3 -31  -4  -1  20 -30 -13 -19 -29 -25 -17  20]
 [ 27   9  -9  10  11 -11 -10  23 -26  23 -11  19  11  22   1]
 [ 15 -24 -23   3 -26   8  26  29  30 -18   9  27 -26  15  18]
 [ 25   1 -23  29  21   3  29   5 -10  -7  16  -9  23  29  29]
 [ 22  -6 -15 -31  30  13 -12  -6 -31  -5  16   7  -6  10  13]
 [ -8 -15  27 -23  10  -7 -13  18   2   2  14  -5 -17   5  -6]
 [ 13 -19 -30  -8   8  -4 -11 -27   9  12  16  -4   3 -22  28]
 [-14 -22  -1   3  17   3   9 -15  -8 -22  -8   7 -29   6  25]
 [ 20 -26  28 -18 -27 -22  28 -27   8 -19   5 -24 -31 -12 -18]
 [-10   0  26 -20  15  30   3 -21 -31 -25  -2  -5   5  23 -12]
 [ -3  19   0  14  23   0 -14  20  30  16 -20 -23 -21   7 -24]
 [ 27  18 -14 -17  10  22  -4  22   6 -26  22  24 -12  -5   8]
 [ 13  21  11  26  24   9   3 -26 -10  13  25   4  14 -21  21]]


In [2]:
vartype = 'BINARY'
# If you make a qubo model, you assign 'BINARY' or 'qubo' to variable vartype.
# if you make a Ising model, you assign 'SPIN' or 'ising' to variable vartype.

graph = IsingLib.Model(vartype)

In [3]:
graph.add_terms(H=mat)
graph.compile()

In [4]:
print(graph.matrix)
s, eng = graph.solve()
print(eng)

[[-10. -11.  10. -16.  -8.  -7.  -5.   3.  13. -24.   4.  -5.  -5. -24. -12.]
 [  0.  -6.  20.  -5.   2.  14. -28.  -4.   9.   8.  25. -24.   9.  18.  -4.]
 [  0.   0. -27.  -3. -31.  -4.  -1.  20. -30. -13. -19. -29. -25. -17.  20.]
 [  0.   0.   0.  10.  11. -11. -10.  23. -26.  23. -11.  19.  11.  22.   1.]
 [  0.   0.   0.   0. -26.   8.  26.  29.  30. -18.   9.  27. -26.  15.  18.]
 [  0.   0.   0.   0.   0.   3.  29.   5. -10.  -7.  16.  -9.  23.  29.  29.]
 [  0.   0.   0.   0.   0.   0. -12.  -6. -31.  -5.  16.   7.  -6.  10.  13.]
 [  0.   0.   0.   0.   0.   0.   0.  18.   2.   2.  14.  -5. -17.   5.  -6.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   9.  12.  16.  -4.   3. -22.  28.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0. -22.  -8.   7. -29.   6.  25.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   5. -24. -31. -12. -18.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  -5.   5.  23. -12.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -2

In [5]:
s, eng = graph.solve('bf')
print(s, eng)

[(1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0)] -346.0


In [6]:
mat, const = graph.get_params('da')
da = IsingLib.DaFormat(mat, api_key='/Users/daisuke.oku/da_key_2.json', version=2)
job_id = da.da_solver()
da.get_job(job_id)

In [7]:
da.get_energy(job_id)
state_li = da.get_configuration(job_id)
for state in state_li:
    print(graph.energy(state))

-346.0


In [8]:
# TSP
import pyqubo

city = 5
vartype = 'BINARY'
x = pyqubo.Array.create('x', (city, city), vartype)
tsp_model = IsingLib.Model(vartype)
dis_li = [[0, 10, 8, 8, 4], [10, 0, 6, 11, 7], [8, 6, 0, 14, 8], [9, 11, 14, 0, 6], [4, 7, 8, 6, 0]]
distance = np.zeros((city**2, city**2))
for i in range(city):
    point = (i + 1) % city
    distance[i*city: (i+1)*city, point*city: (point+1)*city] = dis_li
time_const = pyqubo.Sum(0, city, lambda i: (pyqubo.Sum(0, city, lambda j: x[i, j]) - 1) ** 2)
city_const = pyqubo.Sum(0, city, lambda j: (pyqubo.Sum(0, city, lambda i: x[i, j]) - 1) ** 2)
tsp_model.add_terms(H_0=distance, H_1=time_const, H_2=city_const)

In [9]:
np.set_printoptions(edgeitems=7)
A = 10
tsp_model.compile(H_1=A, H_2=A)
print(tsp_model.matrix)

[[-20.  20.  20.  20.  20.  20.  10.   8.   8.   4.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.]
 [  0. -20.  20.  20.  20.  10.  20.   6.  11.   7.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.]
 [  0.   0. -20.  20.  20.   8.   6.  20.  14.   8.   0.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.  20.   0.   0.]
 [  0.   0.   0. -20.  20.   9.  11.  14.  20.   6.   0.   0.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.  20.   0.]
 [  0.   0.   0.   0. -20.   4.   7.   8.   6.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.  20.]
 [  0.   0.   0.   0.   0. -20.  20.  20.  20.  20.  20.  10.   8.   8.   4.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0. -20.  20.  20.  20.  10.  20.   6.  11.   7.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0. -20.  20.  20.   8.   6.  20.  14.   8.   0.   0.  20.   0.   0.   

In [10]:
state, eng = tsp_model.solve(temperature_start=100000, temperature_end=0.001)
print('')
print(state, )
print('')
ans = state.reshape(city, city)
print(ans)


[0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0]

[[0 0 0 1 0]
 [0 0 0 0 1]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]]


In [11]:
mat, const = tsp_model.get_params('da')
da = IsingLib.DaFormat(mat, api_key='/Users/daisuke.oku/da_key_2.json', version=2)
job_id = da.da_solver()
da.get_job(job_id)

In [12]:
da.get_energy(job_id)
state_li = da.get_configuration(job_id)
for state in state_li:
    np_state = np.array(state).reshape(city, city)
    print(np_state, tsp_model.energy(state))

[[0 0 0 1 0]
 [0 0 0 0 1]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]] -76.0
[[0 1 0 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]] -76.0
